<a href="https://colab.research.google.com/github/harshil0217/NFL_Rookie_Comps/blob/main/ProfileEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import auth
from google.cloud import storage
import io
!pip install sentence-transformers
import spacy
!python -m spacy download en_core_web_sm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
auth.authenticate_user()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 94.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
client = storage.Client(project='cmse-381')
bucket = client.get_bucket('stats_draftprospects')
blob = bucket.blob('draft_profiles.csv')
content = blob.download_as_string()
profiles = pd.read_csv(io.BytesIO(content))

In [3]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of Michigan State head coach John L. Sm...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,Brown has played second fiddle to Carnell Will...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"Edwards' father, Stanley, played at Michigan a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,Benson was drafted by the Los Angeles Dodgers ...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,Williams started two games and played in nine ...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,Jefferson is an efficient back who follows and...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Grant Stuard,Stuard is an undersized off-the-ball linebacke...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


In [4]:
#remove Person, ORG, GPE, Month, Year, and Date

spacy.require_gpu()
nlp = spacy.load("en_core_web_sm")

def remove_names(text):
    doc = nlp(text)
    nouns = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "MONTH", "YEAR", "DATE"]]
    for noun in nouns:
        text = text.replace(noun, "")
    return text

profiles['draft_profile'] = profiles['draft_profile'].apply(remove_names)

In [5]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of head coach . attempted only five ...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,has played second fiddle to throughout his c...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"' father, , played at and in the with the a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,was drafted by in the 12th round out of high...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,started two games and played in nine in befo...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,is an efficient back who follows and reads bl...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Grant Stuard,is an undersized off-the-ball linebacker with...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


In [6]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
embeddings = model.encode(profiles['draft_profile'].tolist())
embeddings = pd.DataFrame(embeddings)

In [8]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.229000,0.092038,-0.238546,-0.409410,-0.058383,-0.016810,0.249242,0.363968,-0.107741,-0.225804,...,0.473348,0.145449,0.043603,-0.083850,-0.276102,-0.161348,-0.282411,0.348845,0.007154,-0.115414
1,0.021272,0.008565,-0.089652,-0.326450,-0.029219,0.037837,0.089914,0.360893,-0.165615,-0.196504,...,0.235946,0.220036,-0.281783,0.137540,-0.408251,-0.180408,-0.016990,-0.081286,0.123472,0.229446
2,0.055434,0.270848,0.054209,-0.355493,-0.293430,-0.313267,-0.039004,0.384535,-0.193526,-0.066518,...,0.181869,0.043800,-0.108755,-0.029446,-0.520516,-0.256389,0.223436,0.032742,-0.103025,-0.005952
3,0.174070,0.190206,-0.250027,-0.283158,0.179868,-0.341568,-0.076105,0.421986,-0.307699,-0.275836,...,0.198810,0.194044,-0.086256,-0.144281,-0.602587,-0.484174,-0.014437,0.136916,0.142912,0.184798
4,0.100643,0.032648,0.039130,-0.460844,-0.121436,-0.083630,-0.003938,0.476229,-0.183685,-0.235308,...,0.504657,0.118731,-0.329506,-0.005814,-0.154123,-0.378601,-0.070724,0.215716,-0.128754,0.160209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,0.052194,0.020961,-0.231056,-0.622643,-0.051415,0.019519,0.146785,0.436300,-0.218264,-0.140882,...,0.282141,0.273848,0.092528,-0.246337,-0.423694,-0.138318,-0.060048,0.090502,0.294183,0.313809
4022,0.277277,0.069675,-0.172049,-0.352510,-0.010618,0.037845,0.254506,0.233208,-0.397032,-0.139837,...,0.361406,-0.141242,-0.104414,-0.490236,-0.369577,0.393444,0.069963,0.151578,-0.044283,0.214912
4023,0.146182,0.310207,-0.401805,-0.523034,-0.022931,0.052813,0.184508,0.134071,-0.327474,-0.191664,...,-0.030537,0.302570,-0.313579,-0.352582,-0.517578,0.045618,-0.259288,-0.042809,0.317930,0.280543
4024,-0.037563,0.304234,-0.565313,-0.369520,0.001178,0.269016,0.070882,0.216647,-0.107958,0.125566,...,0.116432,0.538820,-0.374998,0.029613,-0.316286,-0.095240,-0.493841,0.009139,0.075130,0.236417


In [9]:
#return positive description percentage for each draft profile
def get_positive_description_percentage(text):
    sia = SentimentIntensityAnalyzer()
    scores = sia.polarity_scores(text)
    return scores['pos']

profiles['positive_description_percentage'] = profiles['draft_profile'].apply(get_positive_description_percentage)

In [10]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image,positive_description_percentage
0,Alex Smith,Nephew of head coach . attempted only five ...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN,0.118
1,Ronnie Brown,has played second fiddle to throughout his c...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN,0.098
2,Braylon Edwards,"' father, , played at and in the with the a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN,0.149
3,Cedric Benson,was drafted by in the 12th round out of high...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN,0.053
4,Carnell Williams,started two games and played in nine in befo...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN,0.174
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,is an efficient back who follows and reads bl...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,0.062
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,0.147
4023,Grant Stuard,is an undersized off-the-ball linebacker with...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,0.218
4024,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN,0.056


In [ ]:
# get custom similarity matrix with cosine similarity and diff in positive description percentage sentiment, sentiment weighted 20%

def compute_custom_similarity(embeddings_df, sentiment_series, alpha = 0.2):
    similarity_matrix = cosine_similarity(embeddings_df.values)
    sentiment_series = sentiment_series.values.reshape(-1, 1)
    sentiment_sim_matrix = 1 - np.abs(sentiment_series[:, None] - sentiment_series[None, :])

    similarity_matrix = (1 - alpha) * similarity_matrix + alpha * sentiment_sim_matrix

    return similarity_matrix


similarity_matrix = compute_custom_similarity(embeddings, profiles['positive_description_percentage'])




In [ ]:
similarity_df = pd.DataFrame(similarity_matrix, index=profiles['player_name'], columns=profiles['player_name'])

In [ ]:
similarity_df

In [ ]:
similarity_df.sort_values(by = 'Joe Burrow', ascending = False).head(10)

In [ ]:
similarity_df

In [ ]:
joe_shiesty = similarity_df.loc['Joe Burrow']

In [ ]:
joe_shiesty.sort_values(ascending = False).head(10)